In [ ]:
import quandl
import pandas as pd
import numpy as np
from datetime import datetime 
import pyodbc 
from sqlalchemy import create_engine

# ustanawiam połączenie z bazą
SERVER = 'bigmacindex.database.windows.net'
DATABASE = 'bigmacindex'
DRIVER = 'ODBC Driver 17 for SQL Server'

# docelowo folder z credentialami nie byłby ogólno dostępny, ale chciałem żeby skrypt można było uruchomić 
with open('creds/data_writer_creds.txt','r') as f:
    creds = [line.strip() for line in f.readlines()]

USERNAME = creds[0] 
PASSWORD = creds[1] 


DATABASE_CONNECTION = f'mssql://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}?driver={DRIVER}'

engine = create_engine(DATABASE_CONNECTION)
connection = engine.connect()

# pobieram odpowiednie pola z tabeli geographies, żeby wiedzieć jakie geography_id 
# nadać rekordom z nasdaq oraz jakie informacje pobrać
geographies = pd.read_sql("select geography_id, country_code from geographies where active = 1", connection)

with open('creds/api_key.txt') as f:
    api_key = f.read()

# konfiguracja API, upewniam się żeby nie błędu z powodu przekroczenia liczby odpytań 
# pojawiał mi się tylko błąd 429, ale dla pewności wrzucam pełną listę z dokumentacji
retry_status_codes = [429, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511] 
quandl.ApiConfig.api_key = api_key
quandl.ApiConfig.use_retries =True
quandl.ApiConfig.number_of_retries=5
quandl.ApiConfig.max_wait_between_retries=600
quandl.ApiConfig.retry_status_codes=retry_status_codes

# pobieranie danych
nasdaq_data = pd.DataFrame()
for code in geographies['country_code']:
    link = "ECONOMIST/BIGMAC_" + code
    data = quandl.get(link)
    data['country_code']=code
    nasdaq_data=pd.concat([nasdaq_data,data])

# ustawiam kolumnę date_id w takim samym formacie jak w tabeli dates    
nasdaq_data['date_id'] = nasdaq_data.index
nasdaq_data['date_id'] = nasdaq_data['date_id'].apply(lambda x: int(x.strftime("%Y%m%d")))

# dodaje odpowiednie geography_id oraz id
nasdaq_data = nasdaq_data.merge(geographies, on = 'country_code')
nasdaq_data['id'] = nasdaq_data.index + 1

# wybieram odpowiednie kolumny i ładuję tabelę do bazy danych
nasdaq_data = nasdaq_data[['id', 'date_id', 'geography_id', 'local_price', 'dollar_price', 'dollar_ppp']]
nasdaq_data.to_sql(name='bigmac',con=connection,if_exists='append', index=False)

# dodaje indeksy do tabeli
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)
cursor = cnxn.cursor()

cursor.execute('''CREATE INDEX IX_date_id ON bigmac (date_id)''')
cnxn.commit()

cursor.execute('''CREATE INDEX IX_geography_id ON bigmac (geography_id)''')
cnxn.commit()

connection.close()
cursor.close()
cnxn.close()